In [4]:
# !pip install mysql-connector-python
# !pip install pypika --user
#obtener usuarios de twitter
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse

import locale
import time
from datetime import datetime
from threading import Thread
from tqdm import tqdm
import mysql.connector
import pypika
from pypika import Table, Query, MySQLQuery

## Instrucciones para que ande la notebook:
- Tener una db local de mysql corriendo
- Tener el archivo db_conn.json en la ruta ../config/

In [5]:
with open('../config/db_conn.json', 'r') as db_config_file:
    db_config=json.loads(db_config_file.read())
print('Conectando a', db_config['host'])

database_name = db_config['database']

mydb = mysql.connector.connect(
    host=db_config['host'],
    user=db_config['user'],
    password=db_config['password'],
    database=db_config['database'],
    charset=db_config['charset'],
    port=db_config['port']
)
print('Conectado :)')

Conectando a 127.0.0.1
Conectado :)


## Funciones básicas de scrapeo

In [6]:
def get_profile_timeline_header(username, guest_token, bearer_token):
    return {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate",
        "accept-language": "es-US,es;q=0.9,es-419;q=0.8,en;q=0.7,gl;q=0.6,fr;q=0.5",
        "authorization": "Bearer "+bearer_token,
        "origin": "https://twitter.com",
        "referer": "https://twitter.com/"+username,
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "user-agent": "Mozilla/5.0 (Linux; Android 6.0.1; Moto G (4)) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Mobile Safari/537.36",
        "x-guest-token": guest_token,
        "x-twitter-active-user": "yes",
        "x-twitter-client-language": "es"
    }
def get_profile_timeline_params(user_id, count=1000):
    return {
        "include_profile_interstitial_type": 1,
        "include_blocking": 1,
        "include_blocked_by": 1,
        "include_followed_by": 1,
        "include_want_retweets": 1,
        "include_mute_edge": 1,
        "include_can_dm": 1,
        "include_can_media_tag": 1,
        "skip_status": 1,
        "cards_platform": "Web-12",
        "include_cards": 1,
        "include_ext_alt_text": 'true',
        "include_reply_count": 1,
        "tweet_mode": "extended",
        "include_entities": 'true',
        "include_user_entities": 'true',
        "include_ext_media_color": 'true',
        "include_ext_media_availability": 'true',
        "send_error_codes": 'true',
        "simple_quoted_tweet": 'true',
        "include_tweet_replies": 'true', #respuestas
        "userId": user_id,
        "count": count,
        "ext": "mediaStats,highlightedLabel",
        "include_quote_count": 'true'
    }
def get_profile_timeline_url(user_id, count=1000):
    request_params = get_profile_timeline_params(user_id, count)
    return 'https://api.twitter.com/2/timeline/profile/'+str(user_id)+'.json?'+urllib.parse.urlencode(request_params)

In [7]:
def extract_tweet_entities(tweet):
    entities = {
        'user_mentions': False,
        'user_mentions_l':0,
        'hashtags': False,
        'hashtags_l': 0,
        'urls': False,
        'urls_l': 0,
        'media': False,
        'media_l': 0,
    }
    if(not 'entities' in tweet):
        return entities
    en = tweet['entities']
    if('user_mentions' in en):
        entities['user_mentions'] = [
            {
                'tweet_owner': tweet['user_id_str'], 'tweet_id': tweet['id_str'],
                'user_id': m['id_str']
            }
        for m in en['user_mentions']]
        entities['user_mentions_l'] = len(entities['user_mentions'])
    if('hashtags' in en):
        entities['hashtags'] = [
            {
                'tweet_owner': tweet['user_id_str'], 'tweet_id': tweet['id_str'],
                'hashtag': m['text']
            }
        for m in en['hashtags']]
        entities['hashtags_l'] = len(entities['hashtags'])
    if('urls' in en):
        entities['urls'] = [
            {
                'tweet_owner': tweet['user_id_str'], 'tweet_id': tweet['id_str'],
                'url': m['expanded_url']
            }
        for m in en['urls']]
        entities['urls_l'] = len(entities['urls'])
        
    extended = False
    if('extended_entities' in tweet):
        if('media' in tweet['extended_entities']):
            extended = True
            entities['media'] = [
                {
                    'tweet_owner': tweet['user_id_str'], 'tweet_id': tweet['id_str'],
                    'media_id':m['id_str'], 'url':m['media_url_https'], 'type':m['type']
                }
            for m in tweet['extended_entities']['media']]
            entities['media_l'] = len(entities['media'])
            
    if('media' in en and not extended):
        entities['media'] = [
            {
                'tweet_owner': tweet['user_id_str'], 'tweet_id': tweet['id_str'],
                'media_id':m['id_str'], 'url':m['media_url_https'], 'type':m['type']
            }
        for m in en['media']]
        entities['media_l'] = len(entities['media'])
        
    return entities

def extract_tweet_data(tweet):
    try:
        if(len(re.findall(r'^(RT )', tweet['full_text']))):
            return False
        is_quote = tweet['is_quote_status'] if ('is_quote_status' in tweet) and ('quoted_status_id_str' in tweet) else False
        is_reply = 'in_reply_to_status_id_str' in tweet
        source = re.findall(r'>(.+)</a>$', tweet['source'])
        source = source[0] if len(source)==1 else ''
        entities = extract_tweet_entities(tweet)
        return {
            'tweet': {
                'tweet_id': tweet['id_str'],
                'conversation_id': tweet['conversation_id_str'],
                'user_id': tweet['user_id_str'],
                'created_at': datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S'),
                'lang': tweet['lang'],
                'retweet_count': tweet['retweet_count'],
                'favorite_count': tweet['favorite_count'],
                'reply_count': tweet['reply_count'],
                'quote_count': tweet['quote_count'],
                'is_reply': is_reply,
                'replied_id': tweet['in_reply_to_status_id_str'] if is_reply else None,
                'replied_user_id': tweet['in_reply_to_user_id_str'] if is_reply else None,
                'replied_original': tweet['in_reply_to_status_id_str']==tweet['conversation_id_str'] if is_reply else None,
                'is_quote': is_quote,
                'quoted_id': None if not is_quote else tweet['quoted_status_id_str'],
                'full_text': tweet['full_text'],
                'source': source,
                'user_mentions_count': entities['user_mentions_l'],
                'hashtags_count': entities['hashtags_l'],
                'urls_count': entities['urls_l'],
                'media_count': entities['media_l'],
            },
            'entities': {
                'tweet_id':tweet['id_str'],
                'user_mentions': entities['user_mentions'],
                'hashtags': entities['hashtags'],
                'urls': entities['urls'],
                'media': entities['media'],
            }
        }
    except:
        return False

def get_tweets_data(tweets):
    processed = [extract_tweet_data(tweet[1]) for tweet in tweets]
    processed = [tweet for tweet in processed if tweet != False]
    processed_tweets = [tweet['tweet'] for tweet in processed]
    processed_entities = [tweet['entities'] for tweet in processed]

    get_entity = lambda processed_entities, name: [
        tweet[name] for tweet in processed_entities if tweet[name] != False
    ]

    entities_user_mentions = get_entity(processed_entities, 'user_mentions')
    entities_hashtags = get_entity(processed_entities, 'hashtags')
    entities_urls = get_entity(processed_entities, 'urls')
    entities_media = get_entity(processed_entities, 'media')

    
    return processed_tweets, entities_user_mentions, entities_hashtags, entities_urls, entities_media

In [8]:
def _get_user_prop(prop, user):
    if(prop in user):
        return user[prop]
    return None
def _get_user_image(user):
    if('profile_image_url_https' in user):
        return [
            user['profile_image_url_https'].replace('_normal.', '_400x400.'),
            user['profile_image_url_https']
        ]
    return [None, None]

def extract_user_data(user):
    geo_enabled = 'geo_enabled' in user
    verified = 'verified' in user
    pinned_tweet = user['pinned_tweet_ids_str'][0] if len(user['pinned_tweet_ids_str']) else None
    user_image = _get_user_image(user)
    return {
        'user_id': user['id_str'],
        'user_name': user['screen_name'],
        'name': user['name'],
        'description': user['description'],
        'url': _get_user_prop('url', user),
        'created_at': datetime.strptime(user['created_at'], '%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S'),
        'geo_enabled': _get_user_prop('geo_enabled', user),
        'verified': _get_user_prop('verified', user),
        'profile_image_url': user_image[1],
        'profile_image_url_large': user_image[0],
        'followers_count': user['followers_count'],
        'friends_count': user['friends_count'],
        'listed_count': user['listed_count'],
        'favourites_count': user['favourites_count'],
        'statuses_count': user['statuses_count'],
        'media_count': user['media_count'],
        'pinned_tweet': pinned_tweet,
        'tweets_scraped': None,
    }

def get_users_data(users):
    return [extract_user_data(user[1]) for user in users]

In [9]:
def get_bearer_token():
    _rq_header = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'}
    response = requests.get('https://abs.twimg.com/responsive-web/web/main.92eeeb04.js', headers=_rq_header)
    token = re.findall(r's="(AAAAAA[^"]+)"', response.text)[0]
    return token
def get_guest_token():
    _rq_header = {
        'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Host': 'mobile.twitter.com',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        "origin": "https://mobile.twitter.com",
        "referer": "https://mobile.twitter.com",
    }
    s = requests.Session()
    response = s.get('https://mobile.twitter.com/', headers=_rq_header)
    try:
        token = s.cookies.get_dict()['gt']
    except:
        token = re.findall(r'decodeURIComponent\("gt=([^;]+);', response.text)[0]
    return token
def get_id_from_username(username, guest_token, bearer_token):
    request_params = {
        "variables": json.dumps({
            "screen_name": username,
            "withHighlightedLabel": True
        })
    }
    req_url = 'https://api.twitter.com/graphql/-xfUfZsnR_zqjFd-IfrN5A/UserByScreenName?'+urllib.parse.urlencode(request_params)
    _rq_header = {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate",
        "accept-language": "es-US,es;q=0.9,es-419;q=0.8,en;q=0.7,gl;q=0.6,fr;q=0.5",
        'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0',
        'referer': 'https://mobile.twitter.com/'+username,
        'origin': 'https://mobile.twitter.com',
        'authorization': 'Bearer '+bearer_token,
        'x-guest-token': guest_token,
        "sec-fetch-dest": "empty", "sec-fetch-mode": "cors", "sec-fetch-site": "same-site", "x-twitter-active-user": "yes", "x-twitter-client-language": "es"
    }
    response = requests.get(req_url, headers=_rq_header)
    data = json.loads(response.text)
    user_id = data['data']['user']['rest_id']
    return user_id

In [10]:
def test_lang(tweets, desired_lang='es'):
    tweets = [ 1 if t['lang']==desired_lang else 0 for t in tweets ]
    return sum(tweets) / len(tweets) > .5

def search_user_in_list(users, user_id):
    users = [ user for user in users if user['user_id'] == user_id ]
    if(len(users)!=1):
        return False
    return users[0]

def get_profile_timeline_data(username, guest_token, bearer_token, count=10, user_id=False):
    
    if(user_id == False):
        user_id = get_id_from_username(username, guest_token, bearer_token)
    
    headers = get_profile_timeline_header(username, guest_token, bearer_token)
    url = get_profile_timeline_url(user_id, count)
    page_response = requests.get(url, headers=headers)
    
    if(page_response.status_code != 200):
        print(page_response.text)
        return False
    
    response_json = json.loads(page_response.text)
    tweets = response_json['globalObjects']['tweets']
    users = response_json['globalObjects']['users']
    
    processed_users = get_users_data(list(users.items()))
    
    processed_tweets, entities_user_mentions, entities_hashtags, entities_urls, entities_media = get_tweets_data(list(tweets.items()))
    
    lang_es = test_lang(processed_tweets)
    user_data = search_user_in_list(processed_users, user_id)
    
    return  {
        'scraped_user': username,
        'scraped_user_id': user_id,
        'scraped_user_data': user_data,
        'lang_es': lang_es,
        'users': processed_users,
        'tweets': processed_tweets,
        'entities_user_mentions': entities_user_mentions,
        'entities_hashtags': entities_hashtags,
        'entities_urls': entities_urls,
        'entities_media': entities_media
    }

## Funciones de db

In [11]:
def get_cnxn():
    cnxn = mysql.connector.connect(
        host=db_config['host'],
        user=db_config['user'],
        password=db_config['password'],
        database=db_config['database'],
        charset=db_config['charset'],
        port=db_config['port']
    )
    return cnxn

In [12]:
def get_users_by_id(ids, retry=True):
    cnxn = get_cnxn()
    cursor = cnxn.cursor()
    sql = """
        SELECT * FROM users where user_id IN (?)
    """
    cursor.execute(sql, ', '.join(ids))
    rows = cursor.fetchall()
    return rows

def get_db_users(users):
    users = [ tuple(user.values()) for user in users ]
    t = Table('users', database_name)
    c = ('user_id', 'user_name', 'name', 'description', 'url', 'created_at', 'geo_enabled', 'verified', 'profile_image_url', 'profile_image_url_large', 'followers_count', 'friends_count', 'listed_count', 'favourites_count', 'statuses_count', 'media_count', 'pinned_tweet', 'tweets_scraped')
    q = MySQLQuery.into(t).columns(c).insert( *users )
    return 'INSERT IGNORE'+str(q)[6:]

def update_scraped_user(user_id):
    cnxn = get_cnxn()
    users_table = Table('users', database_name)
    sql = MySQLQuery.update(users_table)\
    .set(users_table.tweets_scraped, 1)\
    .set(users_table.tweets_scraped_at, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))\
    .where(users_table.user_id == user_id)
    cursor = cnxn.cursor()
    cursor.execute(str(sql))
    cnxn.commit()
    return True

def insert_db_users(users, user_id=False):
    cnxn = get_cnxn()
    sql = get_db_users(users)
    cursor = cnxn.cursor()
    cursor.execute(sql)
    cnxn.commit()
    if(user_id):
        update_scraped_user(user_id)
    return True

def update_ignore_user(user_id):
    """Marco un usuario para no scrappearlo por algun motivo (ej esta en otro idioma)"""
    cnxn = get_cnxn()
    users_table = Table('users', database_name)
    sql = MySQLQuery.update(users_table).set(users_table.ignore, 1).where(users_table.user_id == user_id)
    cursor = cnxn.cursor()
    cursor.execute(str(sql))
    cnxn.commit()
    
def insert_ignore_user(user):
    """Inserto y marco un usuario para no scrappearlo por algun motivo (ej esta en otro idioma)"""
    cnxn = get_cnxn()
    sql = get_db_users([user])
    cursor = cnxn.cursor()
    cursor.execute(str(sql))
    cnxn.commit()
    update_ignore_user(user['user_id'])
    return True
    
def get_sql_insert_tweets(tweets):
    columns = ('tweet_id', 'conversation_id', 'user_id', 'created_at', 'lang', 'retweet_count', 'favorite_count', 'reply_count', 'quote_count', 'is_reply', 'replied_id', 'replied_user_id', 'replied_original', 'is_quote', 'quoted_id', 'full_text', 'source', 'user_mentions_count', 'hashtags_count', 'urls_count', 'media_count')
    if(list(tweets[0].keys()) != list(columns)):
        raise Exception('Tweets and db columns are not equal.')
    tweets = [ tuple(tweet.values()) for tweet in tweets ]
    table = Table('tweets', database_name)
    sql = MySQLQuery.into(table).columns(columns).insert( *tweets )
    return 'INSERT IGNORE'+str(sql)[6:]

def insert_db_tweets(tweets):
    """Inserto un array de tweets en la db"""
    cnxn = get_cnxn()
    sql = get_sql_insert_tweets(tweets)
    cursor = cnxn.cursor()
    cursor.execute(sql)
    cnxn.commit()
    return True

def insert_db_rt(user_id, tweets):
    """Inserta retweets hechos por un user_id"""
    retweets = [[tweet['user_id'], tweet['tweet_id'], user_id] for tweet in tweets if user_id != tweet['user_id']]
    columns = ('tweet_owner', 'tweet_id', 'user_id')
    table = Table('retweets', database_name)
    sql = MySQLQuery.into(table).columns(columns).insert( *retweets )
    sql = 'INSERT IGNORE'+str(sql)[6:]
    cnxn = get_cnxn()
    cursor = cnxn.cursor()
    cursor.execute(str(sql))
    cnxn.commit()
    return True

def insert_db_hashtags(hashtags):
    """Inserta retweets hechos por un user_id"""
    hashtags = [tuple(hashtag.values()) for hashtags_t in hashtags for hashtag in hashtags_t]
    columns = ('tweet_owner', 'tweet_id', 'hashtag')
    table = Table('hashtags', database_name)
    sql = MySQLQuery.into(table).columns(columns).insert( *hashtags )
    sql = 'INSERT IGNORE'+str(sql)[6:]
    cnxn = get_cnxn()
    cursor = cnxn.cursor()
    cursor.execute(sql)
    cnxn.commit()
    return True

def insert_db_media(medias):
    """Inserta retweets hechos por un user_id"""
    items = [tuple(i.values()) for tweet in medias for i in tweet]
    columns = ('tweet_owner', 'tweet_id', 'media_id', 'url', 'type')
    table = Table('media', database_name)
    sql = MySQLQuery.into(table).columns(columns).insert( *items )
    sql = 'INSERT IGNORE'+str(sql)[6:]
    cnxn = get_cnxn()
    cursor = cnxn.cursor()
    cursor.execute(sql)
    cnxn.commit()
    return True

def insert_db_mentions(mentions):
    """Inserta retweets hechos por un user_id"""
    items = [tuple(i.values()) for tweet in mentions for i in tweet]
    columns = ('tweet_owner', 'tweet_id', 'user_id')
    table = Table('mentions', database_name)
    sql = MySQLQuery.into(table).columns(columns).insert( *items )
    sql = 'INSERT IGNORE'+str(sql)[6:]
    cnxn = get_cnxn()
    cursor = cnxn.cursor()
    cursor.execute(sql)
    cnxn.commit()
    return True

def insert_db_urls(urls):
    """Inserta retweets hechos por un user_id"""
    items = [tuple(i.values()) for tweet in urls for i in tweet]
    columns = ('tweet_owner', 'tweet_id', 'url')
    table = Table('urls', database_name)
    sql = MySQLQuery.into(table).columns(columns).insert( *items )
    sql = 'INSERT IGNORE'+str(sql)[6:]
    cnxn = get_cnxn()
    cursor = cnxn.cursor()
    cursor.execute(sql)
    cnxn.commit()
    return True

## Ejemplos de db
Insertar un grupo de usuarios en la db:
```python
insert_db_users(response['users'])
insert_db_users(response['users'], response['scraped_user_id']) #actualiza el usuario scrappeado
```
Marcar un usuario que no quiero scrappear:
```python
user = response['scraped_user_data'] #json de usuario
insert_ignore_user(user)
```
Guardar tweets:
```python
insert_db_tweets(response['tweets'])
```
Guardar retweets:
```python
user_id = '1234'
insert_db_rt(user_id, response['tweets'])
```
Obtener usuarios ya scrappeados:
```python
get_users_by_id(['134', '4568'])
```
Insertar media:
```python
insert_db_hashtags(response['entities_hashtags'])
insert_db_media(response['entities_media'])
insert_db_mentions(response['entities_user_mentions'])
insert_db_urls(response['entities_urls'])
```

## Ejemplo para obtener usuario:
```python
username = 'ejemplo_sin_arroba'
user_id = '149991703'
count = 500
bearer_token = get_bearer_token()
guest_token = get_guest_token()

get_id_from_username(
    username,
    guest_token,
    bearer_token,
    count,
    user_id #(opcional)
)
```

#### Respuesta:
La respuesta sera un diccionario con los siguientes atributos:
```python
scraped_user, #username scrapeado
scraped_user_id, #user_id scrapeado
scraped_user_data, #data del usuario scrappeado
lang_es, #la mayoría de los tweets están en español
users, #array el usuario + todos los usuarios con los que interactuó
tweets, #array todos los tweets + rt (pueden no ser originales del usuario que solicite)
entities_user_mentions, #array las menciones
entities_hashtags, #array los hashtags utilizados
entities_urls, #array urls utilizadas
entities_media #imagenes/videos
```

In [13]:
user_id = '149991703'
bearer_token = get_bearer_token()
guest_token = get_guest_token()

response = get_profile_timeline_data(
    'alferdez',
    guest_token,
    bearer_token,
    500,
    user_id
)

In [14]:
response['lang_es']

True

### Ahora si a scrapear

In [15]:
bearer_token = get_bearer_token()
guest_token = get_guest_token()
count_scrapped = 0

In [29]:
def save_user_data(response):
    insert_db_users(response['users'], response['scraped_user_id'])
    insert_db_tweets(response['tweets'])
    insert_db_rt(response['scraped_user_id'], response['tweets'])
    insert_db_hashtags(response['entities_hashtags'])
    insert_db_media(response['entities_media'])
    insert_db_mentions(response['entities_user_mentions'])
    insert_db_urls(response['entities_urls'])
    return True

def scrap_save_user(user_name, user_id=False, verbose=False, count=500):
    global count_scrapped
    global bearer_token
    global guest_token
    if(verbose):
        print('------Init:', user_name)
    if((count_scrapped % 10) == 0):
        if(verbose):
            print('RESET_TOKENS')
        count_scrapped = 0
        bearer_token = get_bearer_token()
        guest_token = get_guest_token()
    count_scrapped+=1
    response = get_profile_timeline_data(
        user_name,
        guest_token,
        bearer_token,
        count,
        user_id
    )
    if(not response['lang_es']):
        #No esta en español, me evito scrappear la cuenta a futuro
        if(verbose):
            print('Not spanish', user_name)
        insert_ignore_user(response['scraped_user_data'])
        return False
    #Guardo la data
    if(verbose):
        print('Saving', user_name, '...')
    save_user_data(response)
    if(verbose):
        print('Saved', user_name)
    return response
def get_new_targets():
    sql = """
        SELECT user_id, user_name FROM users where
        `ignore`= 0 and `tweets_scraped` = 0
        order by followers_count desc
    """
    sql2 = """
        SELECT user_id, user_name FROM users where
        `ignore`= 0 and `tweets_scraped` = 0
        ORDER BY scrapped_at asc
        LIMIT 700
    """
    cnxn = get_cnxn()
    cursor = cnxn.cursor()
    cursor.execute(sql2)
    rows = cursor.fetchall()
    return rows

In [29]:
user_targets = [
    'alferdez',
    'Kicillofok',
    'horaciorlarreta',
    'mauriciomacri',
    'CFKArgentina',
    'NicolasdelCano',
    'jlespert',
    'JalperovichOK',
    'RGBasualdo',
    'blancopabloda',
    'SenadoraBlas',
    'estebanbullrich',
    'Oacastillo',
    'eugecatalfamo',
    'juliocobos',
    'LucilaCrexell',
    'alfredodeangeli',
    'norma_durango',
    'silviaEdePerez',
    'espinolacamau',
    'anabelfsagasti',
    'MarioFiad',
    'giacopposilvia',
    'nancysgonzalez',
    'ElOsoLeavy',
    'gugalusto',
    'SenadorLuenzo',
    'Juancmarino',
    'dalmaciomera',
    'Senadora_Olalla',
    'MariabTapia',
    'juanmariopais',
    'Oscar_Parrilli',
    'luisnaidenoff',
    'pilattivergara',
    'claudiojpoggi',
    'ReutemannC',
    'laurmachado',
    'romeroxsalta',
    'sacnun',
    'sapag_silvia',
    'schiavonih',
    'MaggieSolariQ',
    'lupetagliaferri',
    'JorgeTaiana',
    'PameVerasay',
    'Weretilneck',
    'VZimmermannOK'
]

# for i in user_targets:
#     try:
#         scrap_save_user(i, verbose=True)
#     except KeyboardInterrupt:
#         raise
#     except:
#         print('error', i)

In [ ]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

fracciones = 4
cuenta_errores = 0
from joblib import Parallel, delayed
import random

def parascrap(arg):
    global cuenta_errores
    initial_wait = random.random() * 10
    time.sleep(initial_wait)
    for account in arg:
        try:
            scrap_save_user(account[1], account[0] , verbose=True, count=500)
        except KeyboardInterrupt:
            raise
        except:
            print('Error')
            cuenta_errores += 1

            
for i in range(10):
    user_targets = get_new_targets()
    print('Scrapping:', len(user_targets))
    result = Parallel(n_jobs=fracciones, backend="threading")(map(delayed(parascrap),chunkIt(user_targets,fracciones)))

Scrapping: 700
------Init: GrupoCallao
RESET_TOKENS
------Init: _LauraDiMarco
RESET_TOKENS
Saving _LauraDiMarco ...
Saving GrupoCallao ...
------Init: DHumanosBA
------Init: elenacorregido
Saving elenacorregido ...
Saving DHumanosBA ...
Saved GrupoCallao
------Init: RominaCalderaro
Saving RominaCalderaro ...
Saved elenacorregido
------Init: favifrias
Saved _LauraDiMarco
------Init: horacioalonso16
Saving favifrias ...
Saving horacioalonso16 ...
Saved DHumanosBA
------Init: proargentina
Saving proargentina ...
Saved RominaCalderaro
------Init: NEprovinciabsas
Saved horacioalonso16
------Init: betovaldez
Saving NEprovinciabsas ...
Saving betovaldez ...
Saved favifrias
------Init: MarceGonzaDdHh
RESET_TOKENS
Saving MarceGonzaDdHh ...
Saved proargentina
------Init: _CLACSO
Saving _CLACSO ...
Saved MarceGonzaDdHh
------Init: CaroGaillard
Saved betovaldez
------Init: f_castellaneta
Not spanish f_castellaneta
------Init: realDonaldTrump
Saving CaroGaillard ...
Not spanish realDonaldTrump
----

In [28]:
cuenta_errores

122

In [ ]:
# 31.1 GB

new_targets = get_new_targets()
for i in range(len(new_targets)):
    print(i, 'of', len(new_targets))
    j = new_targets[i]
    try:
        scrap_save_user(j[1], j[0] , verbose=True, count=500)
    except:
        print('####### ERROR #######', j)
        
print('SECOND RUN')

new_targets = get_new_targets()
for i in range(len(new_targets)):
    print(i, 'of', len(new_targets))
    j = new_targets[i]
    try:
        scrap_save_user(j[1], j[0] , verbose=True, count=500)
    except:
        print('####### ERROR #######', j)

0 of 1000
------Init: NuevoDiarioSalt
Saving NuevoDiarioSalt ...
Saved NuevoDiarioSalt
1 of 1000
------Init: RaulJalil_ok
Saving RaulJalil_ok ...
Saved RaulJalil_ok
2 of 1000
------Init: marianorinaldi
Saving marianorinaldi ...
Saved marianorinaldi
3 of 1000
------Init: PactoJujuy
Saving PactoJujuy ...
####### ERROR ####### ('964894883910647808', 'PactoJujuy')
4 of 1000
------Init: tomasbravosl
Saving tomasbravosl ...
####### ERROR ####### ('3353323073', 'tomasbravosl')
5 of 1000
------Init: U24noticias
####### ERROR ####### ('76326090', 'U24noticias')
6 of 1000
------Init: laurasprovierok


From cffi callback <function _verify_callback at 0x000001EBC58FBE18>:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt
From cffi callback <function _verify_callback at 0x000001EBC58FB730>:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


####### ERROR ####### ('2977538919', 'laurasprovierok')
7 of 1000
------Init: Carlitoscapelli
RESET_TOKENS
####### ERROR ####### ('986688956585693185', 'Carlitoscapelli')
8 of 1000
------Init: RodriguezSaaN
RESET_TOKENS
####### ERROR ####### ('144565871', 'RodriguezSaaN')
9 of 1000
------Init: BouhidBacha
RESET_TOKENS
Saving BouhidBacha ...
####### ERROR ####### ('897040943618174976', 'BouhidBacha')
10 of 1000
------Init: CriticaSur
####### ERROR ####### ('727378331239366656', 'CriticaSur')
11 of 1000
------Init: ale_turner


From cffi callback <function _verify_callback at 0x000001EBC91F6E18>:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt
From cffi callback <function _verify_callback at 0x000001EBC91F6378>:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


####### ERROR ####### ('171631723', 'ale_turner')
12 of 1000
------Init: DSantaCruzok
####### ERROR ####### ('147663259', 'DSantaCruzok')
13 of 1000
------Init: OlympicCh_es
####### ERROR ####### ('3305597481', 'OlympicCh_es')
14 of 1000
------Init: Josavila761
####### ERROR ####### ('1235349058862149633', 'Josavila761')
15 of 1000
------Init: LibRepublicano


From cffi callback <function _verify_callback at 0x000001EBC91F69D8>:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt
From cffi callback <function _verify_callback at 0x000001EBC91F6BF8>:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


####### ERROR ####### ('776707273', 'LibRepublicano')
16 of 1000
------Init: larepublica_uy
Saving larepublica_uy ...
Saved larepublica_uy
17 of 1000
------Init: AL_CS_14
####### ERROR ####### ('1892716375', 'AL_CS_14')
18 of 1000
------Init: melilobobibolin
####### ERROR ####### ('241765321', 'melilobobibolin')
19 of 1000
------Init: pepescioli
RESET_TOKENS
####### ERROR ####### ('146611282', 'pepescioli')
20 of 1000
------Init: feriadeturismo
RESET_TOKENS
####### ERROR ####### ('745922894', 'feriadeturismo')
21 of 1000
------Init: unioninf
RESET_TOKENS
####### ERROR ####### ('260261657', 'unioninf')
22 of 1000
------Init: DefensorAhora
RESET_TOKENS
Saving DefensorAhora ...
####### ERROR ####### ('780793376329183232', 'DefensorAhora')
23 of 1000
------Init: silcalder
Saving silcalder ...
Saved silcalder
24 of 1000
------Init: DannyVentu
Saving DannyVentu ...
Saved DannyVentu
25 of 1000
------Init: el_pregonero1
Saving el_pregonero1 ...
